# Pulling from RIOT API
## Miles Johnson

### Introduction
#### What i wanted to achieve:
#### - Pull my league of legends data from my history of matches.
#### - Convert them into a Dataframe using pandas
#### - Export as a csv file to a database 

In [1]:
#Api key generated from portal
api_key= 'RGAPI-14f1679a-7502-401a-a1bd-4f2eed76fb2b'

In [2]:
#Url for summonerv4 api
api_url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/archgenesiss'

In [3]:
import requests

In [4]:
#Url for summonerv4 api
api_url = api_url + '?api_key=' + api_key

In [5]:
api_url

'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/archgenesiss?api_key=RGAPI-14f1679a-7502-401a-a1bd-4f2eed76fb2b'

In [6]:
#Api request 
resp = requests.get(api_url)

In [7]:
resp.json()

{'id': 'lYSU-8ZAIOwQrEMAJEju7kzJRzuTORlantB8qAIiafolwvc',
 'accountId': 'pZujYry_Ui1qHDpALtlODyVeFe5521itgzeyK2raisrXdL4',
 'puuid': '0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw',
 'name': 'archgenesiss',
 'profileIconId': 5896,
 'revisionDate': 1695706520665,
 'summonerLevel': 107}

In [8]:
#Match api based on puuid entered in portal
api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw/ids?start=0&count=20'

In [9]:
# Since their are already args in url we have to use & to add an additional one using our api key
api_url = api_url + '&api_key=' + api_key

In [10]:
api_url

'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw/ids?start=0&count=20&api_key=RGAPI-14f1679a-7502-401a-a1bd-4f2eed76fb2b'

In [11]:
#Api request
resp = requests.get(api_url)
matches = resp.json()

In [12]:
matches

['NA1_4783686285',
 'NA1_4783652217',
 'NA1_4765023061',
 'NA1_4764971657',
 'NA1_4763024815',
 'NA1_4762973640',
 'NA1_4762954541',
 'NA1_4759874005',
 'NA1_4759570819',
 'NA1_4759538503',
 'NA1_4758643215',
 'NA1_4758613462',
 'NA1_4758298016',
 'NA1_4758272488',
 'NA1_4758241138',
 'NA1_4427323326',
 'NA1_4427243194',
 'NA1_4427196900',
 'NA1_4426001154',
 'NA1_4425937034']

In [13]:
#specific match api url using matchid
api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4765023061'

In [14]:
api_url = api_url + '?api_key=' + api_key
api_url

'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4765023061?api_key=RGAPI-14f1679a-7502-401a-a1bd-4f2eed76fb2b'

In [15]:
resp = requests.get(api_url)

In [16]:
#Match data
match = resp.json()

In [17]:
match.keys()

dict_keys(['metadata', 'info'])

In [18]:
match['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [19]:
region = 'americas'
api_key = 'RGAPI-14f1679a-7502-401a-a1bd-4f2eed76fb2b'
puuid = '0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw'

In [20]:
# index of my stats in match data
part_index = match['metadata']['participants'].index(puuid)

In [21]:
#function to get match data using region, match_id, and our apikey should give match data
def get_match_data(region, match_id, api_key):
    api_url = (
    "https://" +
    region + 
    ".api.riotgames.com/lol/match/v5/matches/" +
    match_id + 
    "?api_key=" +
    api_key
    )
    resp = requests.get(api_url)  
    data = resp.json()
    return data

In [22]:
import pandas as pd
import numpy as np

In [23]:
region = 'americas'
api_key = 'RGAPI-14f1679a-7502-401a-a1bd-4f2eed76fb2b'
puuid = '0HCqVZu_blcVOJY2j59sgKgO1WpbnsXkcfZpPYcre0SPHn5OX8JBJGdShogeneC1Vm5aXa4fJlZWVw'

In [24]:
#loop through previous matches, get my stats from each match and store it into a dataframe
match_df = pd.DataFrame()
for match_id in matches:
    match_data = get_match_data(region, match_id, api_key)
    part_index = match_data['metadata']['participants'].index(puuid)
    match_ser = pd.Series(match_data['info']['participants'][part_index]).drop('challenges')
    tmp = pd.DataFrame(match_ser)
    match_df = pd.concat([tmp, match_df], axis=1)
    
   

In [25]:
match_df = match_df.T

In [26]:
match_df = match_df.fillna(0)
match_df = match_df.reset_index().drop('index', axis=1)

In [27]:
#Dataframe of my last 20 games and its stats
match_df

,assists,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,...,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,pushPings,subteamPlacement,totalAllyJungleMinionsKilled,totalEnemyJungleMinionsKilled,visionClearedPings
0,19,0,9,0,13728,15,77,Udyr,0,2,...,0,0,0,0,0,0,0,0,0,0
1,4,0,8,2,10937,13,11,MasterYi,0,1,...,0,0,0,0,0,0,0,0,0,0
2,8,0,14,2,10231,13,77,Udyr,0,3,...,0,0,0,0,0,0,0,0,0,0
3,17,2,17,0,31291,18,131,Diana,0,9,...,0,0,0,0,0,0,0,0,0,0
4,13,0,11,0,14768,16,77,Udyr,0,2,...,0,0,0,0,0,0,0,0,0,0
5,13,0,0,2,17542,17,22,Ashe,0,2,...,0,0,0,0,0,0,0,10,0,0
6,3,0,0,0,10872,13,21,MissFortune,0,1,...,0,0,0,0,0,0,0,4,0,0
7,4,0,0,0,11455,14,21,MissFortune,0,3,...,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,14761,16,21,MissFortune,0,3,...,0,0,0,0,0,0,0,0,0,0
9,5,0,0,0,9365,12,21,MissFortune,0,4,...,0,0,0,0,0,0,0,4,0,0


### Finally put my stats into a dataframe and now its time to start storing it into csv files

In [28]:
# list of columns i care about
stats = ['assists', 'kills', 'deaths', 'championName', 'champLevel', 'goldEarned', 'goldSpent', 'damageDealtToObjectives', 'itemsPurchased',
         'longestTimeSpentLiving', 'totalMinionsKilled', 'wardsPlaced', 'visionScore', 'teamPosition', 'totalDamageDealt', 'totalDamageDealtToChampions', 
         'totalDamageTaken', 'totalTimeSpentDead', 'win']
    

In [29]:
#updating the frame to only those columns
match_df = match_df[stats]

In [30]:
#Where i want my games to start
match_df = match_df.iloc[10:].reset_index(drop=True)

In [31]:
match_df

,assists,kills,deaths,championName,champLevel,goldEarned,goldSpent,damageDealtToObjectives,itemsPurchased,longestTimeSpentLiving,totalMinionsKilled,wardsPlaced,visionScore,teamPosition,totalDamageDealt,totalDamageDealtToChampions,totalDamageTaken,totalTimeSpentDead,win
0,5,4,6,Caitlyn,12,8718,8075,837,12,852,144,6,11,BOTTOM,95927,12396,17535,176,False
1,15,12,5,Tristana,16,14730,14625,13560,21,763,144,11,30,BOTTOM,127642,25186,24512,193,True
2,8,5,1,Caitlyn,12,9923,9600,13674,14,965,155,6,9,BOTTOM,103820,14711,6569,30,True
3,0,2,0,Caitlyn,7,3692,1875,0,4,0,78,1,1,BOTTOM,28461,3511,2324,0,True
4,10,2,6,Caitlyn,16,11094,10425,11839,18,606,187,9,21,BOTTOM,137092,10799,19064,181,False
5,0,1,3,Tristana,8,4611,3625,993,12,728,79,2,2,BOTTOM,27885,3586,6524,48,False
6,15,17,6,Caitlyn,18,17102,15850,15263,22,684,160,9,30,BOTTOM,182193,33548,24615,189,True
7,8,5,9,Lucian,17,14583,13825,10846,28,857,199,8,17,BOTTOM,210940,17073,31623,303,False
8,7,5,6,Aatrox,14,9407,10625,2271,22,790,123,6,11,TOP,88785,12104,25916,243,False
9,14,3,2,Sivir,15,11272,10100,13825,22,874,168,10,25,BOTTOM,143786,20541,13472,38,True


In [36]:
import os
match_df.to_csv(os.path.join("datasets", "match_clean.csv"))